In [25]:
import pandas as pd
import numpy as np

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

In [36]:
sc = pyspark.SparkContext()
ss = SparkSession(sc)

In [7]:
# ingest a file and show it
fil = './Exercise Files/06_02 commands.txt'
data = sc.textFile(fil)
print(type(data))
print('%d records'%data.count())
print('%s'%data.collect())

<class 'pyspark.rdd.RDD'>
26 records
['For Mac:', '', '1) Set path for Spark and adjust to the correct path to Spark', 'echo “export PATH=$PATH:/path_to_downloaded_spark/spark-1.6.0/bin” >> .profile', '', '2) Setting up PySpark driver', 'echo “export PYSPARK_DRIVER_PYTHON=ipython” >> .profile', '', '3) Specifies PySpark options', 'echo “export PYSPARK_DRIVER_PYTHON_OPTS=‘notebook’ pyspark” >> .profile', '', 'For Windows', '1) Puts Spark in a path variable so you can execute commandlets from that directory.  Adjust to your own directory.', '$env:Path += “;C:\\Users\\bsullins\\spark\\bin\\”', '', '2) Set vars for PySpark', '$env:PYSPARK_DRIVER_PYTHON=“ipython”', '', '3) Specifies PySpark options', '$env:PYSPARK_DRIVER_PYTHON_OPTS=“notebook”', '', 'Enter in the following commands instead of source .profile', 'cd ../', 'cd .\\notebooks\\', 'pyspark', '']


In [8]:
print(data.first())
print(data.take(10))

For Mac:
['For Mac:', '', '1) Set path for Spark and adjust to the correct path to Spark', 'echo “export PATH=$PATH:/path_to_downloaded_spark/spark-1.6.0/bin” >> .profile', '', '2) Setting up PySpark driver', 'echo “export PYSPARK_DRIVER_PYTHON=ipython” >> .profile', '', '3) Specifies PySpark options', 'echo “export PYSPARK_DRIVER_PYTHON_OPTS=‘notebook’ pyspark” >> .profile']


In [10]:
# load the data cached into memory so many ops can be performed in memory without disk IO
dataC = sc.textFile(fil).cache()
# count the number of rows that have "spark"
sparkRow = dataC.filter(lambda r: 'spark' in r.lower())
sparkCnt = sparkRow.count()
print(sparkCnt)

13


In [51]:
# ingest all state csv files
fils = './Exercise Files/state_info_*.csv'
files = sc.wholeTextFiles(fils)
print(type(files))
print(files.count())

print(type(files.first()))
print(files.first()[0])
print(files.first()[1])

<class 'pyspark.rdd.RDD'>
2
<class 'tuple'>
file:/C:/Users/howean/Documents/spark_learning/Exercise Files/state_info_AM.csv
State,StateCode,Capital,LargestCity,PopulationEst2015,HouseSeats,StateHood
Alabama,AL,Montgomery,Birmingham,4858979,7,1819-12-14
Alaska,AK,Juneau,Anchorage,738432,1,1959-01-03
Arizona,AZ,Phoenix,Phoenix,6828065,9,1912-02-14
Arkansas,AR,Little Rock,Little Rock,2978204,4,1836-06-15
California,CA,Sacramento,Los Angeles,39144818,53,1850-09-09
Colorado,CO,Denver,Denver,5456574,7,1876-08-01
Connecticut,CT,Hartford,Bridgeport,3590886,5,1788-01-09
Delaware,DE,Dover,Wilmington,945934,1,1787-12-07
Florida,FL,Tallahassee,Jacksonville,20271272,27,1845-03-03
Georgia,GA,Atlanta,Atlanta,10214860,14,1788-01-02
Hawaii,HI,Honolulu,Honolulu,1431603,2,1959-08-21
Idaho,ID,Boise,Boise,1654930,2,1890-07-03
Illinois,IL,Springfield,Chicago,12859995,18,1818-12-03
Indiana,IN,Indianapolis,Indianapolis,6619680,9,1816-12-11
Iowa,IA,Des Moines,Des Moines,3123899,4,1846-12-28
Kansas,KS,Topeka,Wi

In [56]:
# no longer works?
#filesDF = files.toDF(['name','data'])
filesDF = ss.createDataFrame(files, ['name','data']) # the above is a shortcut for this
print(type(filesDF))
display(filesDF) # display only shows the schema if collect or show not included
display(filesDF.collect())

<class 'pyspark.sql.dataframe.DataFrame'>


DataFrame[name: string, data: string]

[Row(name='file:/C:/Users/howean/Documents/spark_learning/Exercise Files/state_info_AM.csv', data='State,StateCode,Capital,LargestCity,PopulationEst2015,HouseSeats,StateHood\r\nAlabama,AL,Montgomery,Birmingham,4858979,7,1819-12-14\r\nAlaska,AK,Juneau,Anchorage,738432,1,1959-01-03\r\nArizona,AZ,Phoenix,Phoenix,6828065,9,1912-02-14\r\nArkansas,AR,Little Rock,Little Rock,2978204,4,1836-06-15\r\nCalifornia,CA,Sacramento,Los Angeles,39144818,53,1850-09-09\r\nColorado,CO,Denver,Denver,5456574,7,1876-08-01\r\nConnecticut,CT,Hartford,Bridgeport,3590886,5,1788-01-09\r\nDelaware,DE,Dover,Wilmington,945934,1,1787-12-07\r\nFlorida,FL,Tallahassee,Jacksonville,20271272,27,1845-03-03\r\nGeorgia,GA,Atlanta,Atlanta,10214860,14,1788-01-02\r\nHawaii,HI,Honolulu,Honolulu,1431603,2,1959-08-21\r\nIdaho,ID,Boise,Boise,1654930,2,1890-07-03\r\nIllinois,IL,Springfield,Chicago,12859995,18,1818-12-03\r\nIndiana,IN,Indianapolis,Indianapolis,6619680,9,1816-12-11\r\nIowa,IA,Des Moines,Des Moines,3123899,4,1846-12-28

In [66]:
display(filesDF.select('name').collect())

[Row(name='file:/C:/Users/howean/Documents/spark_learning/Exercise Files/state_info_AM.csv'),
 Row(name='file:/C:/Users/howean/Documents/spark_learning/Exercise Files/state_info_NZ.csv')]

In [35]:
sc.stop()